In [1]:
import requests
import urllib3
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import os.path
%matplotlib inline
import bs4
import os
print(bs4.__version__)

4.6.3


In [0]:
#@title save your data to drive
filename = "2000-1-1 to 2001-1-1" #@param {type:"string"}
folders_or_files_to_save = "2000-1-1 to 2001-1-1" #@param {type:"string"}
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(path, 
                  mimetype='application/octet-stream',
                  resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created


extension_zip = ".zip"

zip_file = filename + extension_zip

# !rm -rf $zip_file
!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)

In [0]:
company_listing = "http://www.nepalstock.com.np/company?_limit=500"

In [0]:
http = urllib3.PoolManager()
http.addheaders = [('User-agent', 'Mozilla/61.0')]
web_page = http.request('GET',company_listing)
soup = BS(web_page.data, 'html5lib')
table = soup.find('table')
company=[]
rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
col = 0
notfirstrun = False
for row in rows:
    companydata =[]
    for data in row:
        if col == 5 and notfirstrun:
            companydata.append(data.a.get('href').split('/')[-1])
        else:
            companydata.append(data.text.strip())
        col += 1
    company.append(companydata)
    col =0
    notfirstrun = True

df = pd.DataFrame(company[1:],columns=company[0])
df.rename(columns={'Operations':'Symbol No'},inplace=True)
df.index.name = "SN"
df.drop(columns='',inplace=True)
df.drop(columns='S.N.',inplace=True)
df.to_json('CompanyList.json',orient='index')
print('There are %s Companies'%len(df.index))
df.head()
df.to_csv('CompanyList.csv', encoding='utf-8', index=False)

There are 220 Companies


In [0]:
# Getting Company Details
symbol = "NMB"
url = "http://www.nepalstock.com/company/"
try:
    req = requests.post(url, data={"stock_symbol":symbol}, verify=False)
except requests.exceptions.RequestException as e:
    print(e)
response = req.text
soup = BS(response, "lxml")
table = soup.find("table")
print ("Company: ",table.findAll("td")[0].string)
for row in table.findAll("tr")[4:]:
    col = row.findAll("td")
    print (col[0].string,": ",col[1].string)

Company:  NMB Bank Limited
Address :  Babarmahal
Email Address :  pramod.dahal@nmb.com.np
Website :  http://www.nmb.com.np
Last Traded Price (Rs.) :  331.00
Change (Rs.) and (%) :  3(0.91)
Total Listed Shares :  87,437,842.00
Paid Up Value (Rs.) :  100
Total Paid Up Value (Rs.) :  8,743,784,200.00
Closing Market Price (Rs.) :  331.00
Market Capitalization (Rs.) :  28,941,925,702.00


In [0]:
DailyFloorSheet="http://www.nepalstock.com.np/main/floorsheet/index/0/?_limit=5000"

http = urllib3.PoolManager()
http.addheaders = [('User-agent', 'Mozilla/61.0')]
web_page = http.request('GET',DailyFloorSheet)
soup = BS(web_page.data, 'html5lib')
table = soup.find('table')
FloorSheet=[]
rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
for row in rows:
    FloorSheet.append([data.text.strip() for data in row])
FloorSheetdf = pd.DataFrame(FloorSheet[1:-1],columns=FloorSheet[0])
FloorSheetdf.head()

,S.N.,Contract No,Stock Symbol,Buyer Broker,Seller Broker,Quantity,Rate,Amount
0,1,2019040201007824,LBL,5,40,35,200,7000.00
1,2,2019040201007823,LBL,7,40,1000,201,201000.00
2,3,2019040201007822,LBL,34,40,965,201,193965.00
3,4,2019040201007821,LBL,58,40,1000,201,201000.00
4,5,2019040201007820,NBB,37,26,56,204,11424.00


In [0]:
df['date'] = pd.to_datetime(FloorSheetdf['Contract No'],format='%Y%m%d%H%M%f').dt.date

In [0]:
def CompanyStocksTransactions(SymbolNo,startDate, endDate):
    url="http://www.nepalstock.com.np/company/transactions/%s/0/?startDate=%s&endDate=%s&_limit=9000000"%(SymbolNo,startDate, endDate)
    #print("Connecting to %s "%url)
    http = urllib3.PoolManager()
    http.addheaders = [('User-agent', 'Mozilla/61.0')]
    web_page = http.request('GET',url)
    #print("Adding to DataFrame")
    soup = BS(web_page.data, 'html5lib')
    table = soup.find('table')
    FloorSheet=[]
    rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
    for row in rows:
      FloorSheet.append([data.text.strip() for data in row])
    if(len(FloorSheet) != 0):
      FloorSheetdf = pd.DataFrame(FloorSheet[1:],columns=FloorSheet[0])
      FloorSheetdf['Date']=pd.to_datetime(FloorSheetdf['Contract No'], format='%Y%m%d%H%M%f', errors='ignore')
      return (1, FloorSheetdf)
    else:
      return (0, None)

In [0]:
os.mkdir('hi')


In [0]:
for year in range(20):
  startDate = 2000 + year
  startDate = str(startDate) + '-1-1'
  
  endDate = 2001 + year
  endDate = str(endDate) + '-1-1'
  
  count = 0
  print("From year %s to %s "%(startDate, endDate))
  dir_name = startDate + " to " + endDate
  os.mkdir(dir_name)
  print("Directory created\n")
  for symbol in list(df['Symbol No']):
      #print("Stock No: %s\nTime: %s"%(symbol,time.ctime()))
      filename = dir_name + "/NEPSE%s.csv"%symbol
      count +=1
      if os.path.isfile(filename):
          print("Stock No: %s Data Already Downloaded"%symbol)
      else:
          time1 = time.time()
          success, dftest=CompanyStocksTransactions(symbol,startDate, endDate)
          if(success != 0):
            dftest.to_csv(filename, encoding='utf-8', index=False)
            #print("%s of %s completed in %s \n"%(count,len(df['Symbol No']), time.time() - time1))
          #else:
            #print('Error....')
  print("Completed this year. \n")

From year 2000-1-1 to 2001-1-1 
Directory created



In [0]:
# plot
# Need to convert Contract Number to Date-time
Stock = 'EBL'
symbol = "%d"%df[df['Stock Symbol']==Stock]['Symbol No']
print(symbol)
dfNepse=pd.read_csv("./data/NEPSE%s.csv"%symbol)
dfNepse.sort_values('Contract No',inplace = True)
startfrom=200101016341249
dfNepse=dfNepse[dfNepse['Contract No']>startfrom]
dfNepse.plot(x='Contract No', y = 'Rate')

In [0]:
dfNepse.describe()

In [0]:
symbol = "NMB"
url = "http://nepalstock.com/marketdepth/"
try:
    req = requests.post(url, data={"stock_symbol":symbol}, verify=False)
except requests.exceptions.RequestException as e:
    print(e)
response = req.text
soup = BS(response, "lxml")
tables = soup.findAll("table")
for table in tables:
    for row in table.findAll("tr")[4:]:
        col = row.findAll("td")
        print (col[0].string,": ",col[1].string)